### action 处理

In [28]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

#读取量化的参与意愿
activity = pd.read_csv('./row_data/action_willingness.csv', index_col='enroll_id')

#读取最初的
data = pd.read_csv('./row_data/number_data.csv',
                   index_col='enroll_id',
                   usecols=[0, 21, 22, 23],
                   )

activity = pd.merge(data, activity, on='enroll_id', how='outer')

activity.to_csv('./action_data_process/willingness_merge.csv')


In [29]:
activity = pd.read_csv('./action_data_process/willingness_merge.csv')
del activity['enroll_id']
activity.to_csv('./action_data_process/model_input_willingness.csv',index=0)

### action划分训练集和测试集

In [58]:
#只按照7：3，8：2的比例去划分

import random
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold
class DataSet():
    def __init__(self, basedir):
        self.basedir = basedir
        read_dir = basedir + '/action_data_process/'
        save_dir = basedir + '/action_data_process/'
        data = pd.read_csv(read_dir + "model_input_willingness.csv")

        self.record = data.set_index('username')
        self.total_stu_list = list(set(self.record.index))
        self.save_dir = save_dir

    def get_train_test(self, test_ratio = 0.2):
            train, test= split_record(self.record, test_ratio= test_ratio)
            return train, test  # 这里划分了， index_loader里的id应该从这里取


def split_record(record, test_ratio = 0.2):
    total_stu_list = list(set(record.index))
    train_data = [[], [], [], []]
    test_data = [[], [], [], []]
    for stu in total_stu_list:
        stu_data = record.loc[stu, :]
        stu_cour = np.array(stu_data['course_id'])
        stu_truth = np.array((stu_data['truth']))
        stu_activity = np.array(stu_data['willingness'])
        length = len(stu_cour)

    
        index_list = list(range(length))
 
        test_index = random.sample(index_list, math.ceil(length * test_ratio))

        #剩下的所有为训练集
        train_index = list(set(index_list) - set(test_index))

        train_data[0].extend([stu] * len(train_index))
        train_data[1].extend(stu_cour[train_index])
        train_data[2].extend(stu_truth[train_index])
        train_data[3].extend(stu_activity[train_index])


        test_data[0].extend([stu] * len(test_index))
        test_data[1].extend(stu_cour[test_index])
        test_data[2].extend(stu_truth[test_index])
        test_data[3].extend(stu_activity[test_index])

    train = pd.DataFrame({'username': train_data[0], 'course_id': train_data[1],'truth': train_data[2],'willingness': train_data[3]},).set_index('username')
    test = pd.DataFrame({'username': test_data[0], 'course_id': test_data[1], 'truth': test_data[2],'willingness': test_data[3]},).set_index('username')
    return train, test

if __name__ == '__main__':
    # ----------基本参数--------------
    basedir = 'D:/pythonProject/firstpaper/firstdata_process'
    dataSet = DataSet(basedir)
    save_dir = dataSet.save_dir
    test_ratio = 0.2
    train_data, test_data = dataSet.get_train_test(test_ratio= test_ratio)
    print(len(train_data))
    print(len(test_data))
    train_data.to_csv(save_dir+'train_data_2.csv')
    test_data.to_csv(save_dir+'test_data_2.csv')
    # ----------基本参数--------------



69171
72694


In [59]:
train_data = pd.read_csv('./action_data_process/train_data_2.csv')
test_data = pd.read_csv('./action_data_process/test_data_2.csv')

del train_data['truth']
del test_data['truth']

train_data.to_csv('./model_input_data/model_train_data_2.csv',index=0)
test_data.to_csv('./model_input_data/model_test_data_2.csv',index=0)



### 利用action进行选课邻接矩阵处理

In [1]:
import pandas as pd

In [46]:
adj2_data = pd.read_csv('./model_input_data/model_train_data_2.csv',usecols = [0,1])
df = pd.crosstab(adj2_data.username,adj2_data.course_id)

df.to_csv('./model_input_data/adjacency_2.csv',)

### 利用action求聚合矩阵

In [91]:
martix_data = pd.read_csv('./model_input_data/model_train_data_2.csv',usecols = [0,1])
row_data = pd.read_csv('./row_data/number_data.csv',usecols = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])
data = pd.merge(martix_data, row_data, on=['username','course_id'], how='inner')
data.to_csv('./action_martix/train_data_2.csv')

### 分别对用户和课程进行分类，用来求聚合矩阵

In [92]:

import csv
result = {}
if __name__ == '__main__':
    switch = 0
    header = []
    with open("./action_martix/train_data_2.csv") as f:
        csv_reader = csv.reader(f)
        for row in csv_reader:
            if switch == 0:
                header = row
                switch = 1
                continue
            if row[1] not in result.keys():
                result[row[1]] = []
                result[row[1]].append(header)
                result[row[1]].append(row)
            else:
                result[row[1]].append(row)
        f.close()

    for file_name in result.keys():
        with open("./action_martix/martix_2_sort_user/"+file_name+".csv", 'w', newline="") as f:
            writer = csv.writer(f)
            writer.writerows(result[file_name])
            f.close()




In [93]:
import csv
result = {}
if __name__ == '__main__':
    switch = 0
    header = []
    with open("./action_martix/train_data_2.csv") as f:
        csv_reader = csv.reader(f)
        for row in csv_reader:
            if switch == 0:
                header = row
                switch = 1
                continue

         
            if row[2] not in result.keys():
                result[row[2]] = []
                result[row[2]].append(header)
                result[row[2]].append(row)
            else:
                result[row[2]].append(row)
        f.close()

    for file_name in result.keys():
        with open("./action_martix/martix_2_sort_course/"+file_name+".csv", 'w', newline="") as f:
            writer = csv.writer(f)
            writer.writerows(result[file_name])
            f.close()

### 对行为求均值

In [94]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './action_martix/martix_2_sort_user/*.csv'
all_files = glob.glob(input_path)

for file in all_files:
    data_frame = pd.read_csv(file,
                             index_col=None,
                             # usecols=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                             encoding='gbk')
    key_cols = data_frame.keys().tolist()[3:22]
    data_frame[key_cols] = data_frame[key_cols].apply(
        lambda x: x.mean()
    )

    
    file_name = file.split('\\')[-1]  
    data_frame.to_csv('./action_martix/mean_2_sort_user/' + file_name, index=False)


In [95]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './action_martix/martix_2_sort_course/*.csv'
all_files = glob.glob(input_path)
for file in all_files:
    data_frame = pd.read_csv(file,
                             index_col=None,
                             # usecols=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                             encoding='gbk')
    key_cols = data_frame.keys().tolist()[3:22]
    data_frame[key_cols] = data_frame[key_cols].apply(
        lambda x: x.mean()
    )
    file_name = file.split('\\')[-1] 
    data_frame.to_csv('./action_martix/mean_2_sort_course/' + file_name, index=False)

### 求聚合

In [ ]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './action_martix/mean_2_sort_user/*.csv'

# glob.glob()函数将会匹配给定路径下的所有pattern，并以列表形式返回
all_files = glob.glob(input_path)
all_data = pd.DataFrame()

# 循环遍历每个文件
for file in all_files:
    data_frame = pd.read_csv(file,
                             usecols=[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
                             index_col=None,
                             encoding='gbk')
    data = data_frame.iloc[0]

    all_data = all_data.append(data)
    # all_data.set_index('course_id',inplace = True)
# print(all_data)
all_data.to_csv('./model_input_data/avg_course_2.csv',index=False)

In [ ]:
import pandas as pd
import glob
import csv
import numpy as np

input_path = './action_martix/mean_2_sort_course/*.csv'

# glob.glob()函数将会匹配给定路径下的所有pattern，并以列表形式返回
all_files = glob.glob(input_path)
all_data = pd.DataFrame()

# 循环遍历每个文件
for file in all_files:
    data_frame = pd.read_csv(file,
                             usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
                             index_col=None,
                             encoding='gbk')
    data = data_frame.iloc[0]

    all_data = all_data.append(data)
    # all_data.set_index('course_id',inplace = True)
# print(all_data)
all_data.to_csv('./model_input_data/avg_user_2.csv',index=False)